<a href="https://colab.research.google.com/github/alexcpn/tranformer_learn/blob/main/gpt2_overfitting_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.28.1
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.7 MB/s eta 0:00:00


In [11]:
#upload files to your colab environment
!wget https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
#!wget https://gist.githubusercontent.com/alexcpn/54e88130f9d186494f1c3ce5e83263b4/raw/7cdf5f93b819024c58a891fc808fbdbe052d0eb1/small_3_mixed.txt

--2023-06-16 19:31:39--  https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56513 (55K) [text/plain]
Saving to: ‘small_3.txt.4’

small_3.txt.4       100%[===================>]  55.19K  --.-KB/s    in 0.002s  

2023-06-16 19:31:39 (31.4 MB/s) - ‘small_3.txt.4’ saved [56513/56513]



In [12]:
train_path = 'small_3.txt'


In [13]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator

tokenizer = AutoTokenizer.from_pretrained("gpt2")
train_dataset,data_collator = load_dataset(train_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-small3-v4", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=500, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
)

In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.710500
1000,0.326400
1500,0.044900
2000,0.025800
2500,0.022200
3000,0.019300
3500,0.017200
4000,0.016000
4500,0.015600
5000,0.013800


TrainOutput(global_step=12000, training_loss=0.13877874926726022, metrics={'train_runtime': 2150.8332, 'train_samples_per_second': 21.62, 'train_steps_per_second': 5.579, 'total_flos': 3037519872000000.0, 'train_loss': 0.13877874926726022, 'epoch': 500.0})


Short detail about training -
- Step	Training Loss
- 500	2.263500
- 1000	0.205800
- 1500	0.029900

Took about 6 hours in Colab Free in TPU runtime

Stopped at Epoch 35 as the Training loss was pretty low

1655/2350 7:15:15 < 3:03:00, 0.06 it/s, Epoch 35.19/50]

In [16]:
trainer.save_model()

# Test Model

In [17]:
!zip -r gpt2-small3-v4.zip gpt2-small3-v4/config.json  gpt2-small3-v4/training_args.bin  gpt2-small3-v4/pytorch_model.bin gpt2-small3-v4/generation_config.json


  adding: gpt2-small3-v4/config.json (deflated 51%)
  adding: gpt2-small3-v4/training_args.bin (deflated 48%)
  adding: gpt2-small3-v4/pytorch_model.bin (deflated 9%)
  adding: gpt2-small3-v4/generation_config.json (deflated 24%)


In [18]:
!cp gpt2-small3-v4.zip ./drive/MyDrive/models

In [2]:
!cp ./drive/MyDrive/models/gpt2-small3-v4.zip .

In [3]:
!unzip gpt2-small3-v4.zip

Archive:  gpt2-small3-v4.zip
  inflating: gpt2-small3-v4/config.json  
  inflating: gpt2-small3-v4/training_args.bin  
  inflating: gpt2-small3-v4/pytorch_model.bin  
  inflating: gpt2-small3-v4/generation_config.json  


In [4]:
from transformers import pipeline

test = pipeline('text-generation',model='./gpt2-small3-v4/', tokenizer='gpt2')

In [12]:
test('An alkaline medium favours', max_new_tokens=512)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes. The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes. The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes. The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes. The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes. The great majority

This is as  expected from the passage
[ An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes . The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes

In [21]:
test('Streptococci are met with in', max_new_tokens=120,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character. Streptococci are met with in erysipelas and various other inflammatory processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 2). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile. Some forms are non-motile. Some forms are motile, others are non-motile. Some forms are motile, others are non-'},
 {'generated_text': 'Streptococci are met with in erysipelas and various other inflammatory and suppurative processes. Streptococci are met with in er comparatively short periods, seldom longer than a few days. Streptococci may be Streptococci of a Streptococcus  or of a Streptococcus  although the two are so intimately related that it is impossible to draw a hard-and-fast line between them, but for convenience Streptococci may be grouped in

The inital part is exaclty as in the passage [Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 4). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile.]

The last line

from passage [Tuberculosis, tetanus, anthrax, and many other surgical diseases are due to different forms of bacilli.  Spirilla  are long, slender, thread-like cells, more or less spiral or wavy.]

The last line generated is not correct

"Some forms are motile only in virtue of the contractility of the protoplasm, "

is there in another pace where "motile" is reffered

whereas the following is pure hallicunation

"some in virtue of the fibroblasts which they carry. Others are'}]"

In [23]:
test('Streptococci', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Streptococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig'},
 {'generated_text': 'Streptococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig'}]

In [25]:
test('Metchnikoff', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Metchnikoffylococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are'},
 {'generated_text': 'Metchnikoff, which are similar to those of the phagocytes. In the course of operations also, the serum or serum may be interfered with by various extraneous agencies, the most important of which are infection by disease-producing micro-organisms, the presence of foreign substances, undue movement of the affected part, and improper applications and dressings. The effect of these agencies is to delay repair or to prevent the individual organisms carrying the process of repair. Metchnikoff also indicates that these agencies

In [27]:
test('To this process Metchnikoff', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'To this process Metchnikoff gave the name of  phagocytosis, and he recognised two forms of  phagocytes : (1) the  microphages, which are the polymorpho-nuclear leucocytes of the blood; and (2) the  macrophages, which include the larger hyaline leucocytes, endothelial cells, and connective-tissue corpuscles. During the process of phagocytosis, the polymorpho-nuclear leucocytes in the circulating blood'},
 {'generated_text': 'To this process Metchnikoff gave the name of  phagocytosis, and he recognised two forms of  phagocytes : (1) the  microphages, which are the polymorpho-nuclear leucocytes of the blood; and (2) the  macrophages, which include the larger hyaline leucocytes, endothelial cells, and connective-tissue corpuscles. During the process of phagocytosis, the polymorpho-nuclear leucocytes in the circulating blood'}]

In [28]:
test('Phagocytosis', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Phagocytosis ), as well as in their phagocytic action, and in the course of destroying the bacteria they produce certain ferments which enter the blood serum. These are known as  opsonins  or  alexins, and they act on the bacteria by a process comparable to narcotisation, and render them an easy prey for the phagocytes. Runic and Pain.  When a wound is made in the integument under aseptic conditions, the passage of the knife'},
 {'generated_text': 'Phagocytosis ), as well as in their phagocypurpose, and in the course of destroying the bacteria they produce certain ferments which enter the blood serum. These are known as  opsonins  or  alexins, and they act on the bacteria by a process comparable to narcotisation, and render them an easy prey for the phagocytes.  Artificial or Passive Immunity.  A form of immunity can be induced by the introduction of protective substances obtained from an animal which has been actively'}]

In [29]:
test('During the process of phagocytosis,', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'During the process of phagocytosis, the scar tissue covering the superficial parts of the wound becomes permeated by granulation tissue, the fibroblasts of which are derived from the sheath of the tendon, the surrounding connective tissue, and probably also from the divided ends of the tendon itself. These fibroblasts ultimately develop into typical tendon cells, and the fibres which they form constitute the new tendon fibres. Under aseptic conditions repair is complete in from two to three weeks. In the course of the rep'},
 {'generated_text': 'During the process of phagocytosis, the scar cells in the vicinity of the wound begin to proliferate, and during the process young connective tissue cells permeate the superficial parts of the wound and temporarily increase its blood supply, but in the consolidation of the new fibrous tissue these vessels are ultimately obliterated. This does not prove that the operation is useless, but it indicates that the process is taki

In [9]:
test(' diplococci ', max_new_tokens=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' diplococci  and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig.'},
 {'generated_text': ' diplococci  and of streptococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3 cleaning off). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice'}]

In [12]:
 test('Cocci  or  micrococci', max_new_tokens=100,num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Cocci  or  micrococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig.onyms). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 2). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig'},
 {'generated_text': 'Cocci  or  micrococci, and to this variety the commonest pyogenic or pus-forming organisms belong (Fig. 2). When division takes place only in one axis, so that long chains are formed, the term  streptococcus  is applied (Fig. 3). Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig'},
 {'generated_text': 'Cocci  or  micrococci, and to this variety t

In [14]:
 test('Bacteria are most conveniently', max_new_tokens=100,num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Bacteria are most conveniently classified according to their shape. Thus we recognise (1) those that are globular  cocci ; (2) those that are not  cocci ; and (3) those that are bacilli. When degradation of the structures of the host body takes place only in a few hours, so that long chains are formed, the term  streptococcus  is applied (Fig. 2). Streptococci are met with in erysipelas and various other inflammatory and supp'},
 {'generated_text': 'Bacteria are most conveniently classified according to their shape. Thus we recognise (1) those that are globular  cocci ; (2) those that are non-motile  cocci ; and (3) those that are bacilli.  General Characters of Bacteria.  A  bacterium  consists of a finely granular mass of protoplasm, enclosed in a thin gelatinous envelope. Many forms are motile some in virtue of fine thread-like flagella, and others through contract'},
 {'generated_text': 'Bacteria are most conveniently classified according to their shape. Thus w